In [4]:
import pandas as pd
import cv2
import numpy as np

In [5]:
# Load images and CSV data
csv_file = 'labels_2024-11-10-03-44-34.csv'
image_folder = 'Images/'

df = pd.read_csv(csv_file)

In [7]:
df.head()

,Person,245,851,1660806000.jpg,1920,1080
0,Person,154,785,1660806000.jpg,1920,1080
1,Person,474,584,1660806000.jpg,1920,1080
2,Person,498,582,1660806000.jpg,1920,1080
3,Person,632,553,1660806000.jpg,1920,1080
4,Person,658,565,1660806000.jpg,1920,1080


In [ ]:
def apply_clahe(image):
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(image_gray)

In [ ]:
def average_images(image_list):
    avg_image = np.zeros_like(image_list[0], np.float32)
    for img in image_list:
        avg_image += img.astype(np.float32) / len(image_list)
    return avg_image.astype(np.uint8)


In [ ]:
def apply_edge_detection(image):
    edges = cv2.Canny(image, 100, 200)
    return edges


In [ ]:
def compare_with_empty(image, empty_image):
    return cv2.absdiff(image, empty_image)


In [ ]:
def apply_morphological_operations(image):
    kernel = np.ones((5,5), np.uint8)
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    return opening


In [ ]:
# Load and process images
empty_image_list = [cv2.imread(f'{image_folder}{img_name}') for img_name in ['empty1.jpg', 'empty2.jpg', ...]]
empty_image = average_images(empty_image_list)

for _, row in df.iterrows():
    image_path = f'{image_folder}{row["image name"]}'
    image = cv2.imread(image_path)
    
    image_clahe = apply_clahe(image)
    image_edges = apply_edge_detection(image_clahe)
    image_difference = compare_with_empty(image_edges, apply_edge_detection(apply_clahe(empty_image)))
    image_morphed = apply_morphological_operations(image_difference)
    
    # Here, you can count contours or use further algorithms to count people.
    contours, _ = cv2.findContours(image_morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(f'Counted people in {row["image name"]}: {len(contours)}')
